In [1]:
from scipy.spatial import KDTree
from geopy.distance import geodesic
import pandas as pd

In [2]:
gh_df = (
    pd.read_csv("geohash_traffic_density_pt_15clusters.csv", usecols=['GEOHASH', 'LATITUDE', 'LONGITUDE'])
)

# Reading Metro data and ensuring to take only unique coordinates
site_dict = (
    pd.read_csv("datasets/08_tr_istanbul_rail_sea_transportation_locations/tr_ist_metro_stns.csv")
    .drop_duplicates(subset=['Latitude', 'Longitude'])
    .assign(Name=lambda x: x['Name'] + "_" + x['LineName'])
    .set_index('Name')
    .T.to_dict()
)

In [3]:
# Unique list of coordinates
coordinates = gh_df[['LATITUDE', 'LONGITUDE']].drop_duplicates().values
coordinates

array([[41.080627, 28.811646],
       [40.987244, 29.108276],
       [41.003723, 29.09729 ],
       ...,
       [40.976257, 29.229126],
       [41.009216, 27.98767 ],
       [41.020203, 27.998657]])

In [4]:
def find_nearest_coordinates(
        target_lat: float,
        target_lon: float,
        coordinates: list[list[float, float]]
        ) -> list[tuple[tuple[float, float], float]]:
    """
    Finds the nearest coordinates to a target latitude and longitude.

    Args:
        target_lat (float): The latitude of the target location.
        target_lon (float): The longitude of the target location.
        coordinates (List[Tuple[float, float]]): A list of tuples representing the coordinates to search from.

    Returns:
        List[Tuple[Tuple[float, float], float]]: A list of tuples containing the nearest coordinates and their distances in kilometers.
    """
    target_coords = (target_lat, target_lon)
    
    # Build a KDTree for efficient nearest neighbor search
    tree = KDTree(coordinates)

    # Calculate the search radius in degrees
    radius_deg = 1 / 111.0  # 1 km radius in degrees

    # Query the KDTree for indices of points within the search radius
    indices = tree.query_ball_point([(target_lat, target_lon)], radius_deg)

    # Calculate the distance between target coordinates and nearest coordinates
    nearest_coordinates = [(coordinates[i], geodesic(target_coords, coordinates[i]).km) for i in indices[0]]

    # Sort the nearest coordinates by distance
    nearest_coordinates.sort(key=lambda x: x[1])  # Sort by distance

    return nearest_coordinates

In [5]:
metro_stn_list = []
new_list = []

for site in site_dict:

    target_lat = site_dict[site]['Latitude']
    target_lon = site_dict[site]['Longitude']

    nearest_coords = find_nearest_coordinates(target_lat, target_lon, coordinates)

    for i, (coords, distance) in enumerate(nearest_coords):

        gh = gh_df.loc[gh_df['LATITUDE'].eq(coords[0]) & gh_df['LONGITUDE'].eq(coords[1]), 'GEOHASH'].values[0]

        # Creating two dataframe which will later be merged rows wise for visualization purpose.
        metro_stn_list.append({
            "type" : "metro_stn",
            "name" : site,
            "latitude" : target_lat,
            "longitude" : target_lon,
            "nearest" : site
        })

        new_list.append({
            "type" : "geohash",
            "name" : gh,
            "latitude" : coords[0],
            "longitude" : coords[1],
            "nearest" : site
        })

In [6]:
df = (
    pd.concat([pd.DataFrame(new_list), pd.DataFrame(metro_stn_list)], ignore_index=True)
    .drop_duplicates(ignore_index=True)
)

df

,type,name,latitude,longitude,nearest
0,geohash,sxk9hq,40.992737,29.020386,KADIKOY_M4
1,geohash,sxk9hm,40.987244,29.020386,KADIKOY_M4
2,geohash,sxk9hr,40.998230,29.020386,KADIKOY_M4
3,geohash,sxk9hx,40.998230,29.031372,AYRILIK CESMESI_M4
4,geohash,sxk9k8,41.003723,29.031372,AYRILIK CESMESI_M4
...,...,...,...,...,...
976,metro_stn,KURTKOY_M4,40.910620,29.297057,KURTKOY_M4
977,metro_stn,SABIHA GOKCEN_M4,40.905974,29.310554,SABIHA GOKCEN_M4
978,metro_stn,RUMELI HISARUSTU_F4,41.084867,29.051021,RUMELI HISARUSTU_F4
979,metro_stn,PARSELLER_M8,41.031879,29.152689,PARSELLER_M8


In [7]:
# Some GEOHASH has up to 11 metro stations nearby
(
    df.query("type=='geohash'")
    .groupby('name')['nearest'].nunique()
    .nlargest(10)
)

name
sxk9hm    11
sxk96c     7
sxk9hq     7
sxk9ht     7
sxk967     6
sxk976     6
sxk977     6
sxk97t     6
sxk97w     6
sxk9dw     6
Name: nearest, dtype: int64

In [9]:
(
    df.query("name=='sxk9hm'")
)

,type,name,latitude,longitude,nearest
1,geohash,sxk9hm,40.987244,29.020386,KADIKOY_M4
695,geohash,sxk9hm,40.987244,29.020386,ISKELE CAMII_T3
700,geohash,sxk9hm,40.987244,29.020386,CARSI_T3
708,geohash,sxk9hm,40.987244,29.020386,BAHARIYE_T3
710,geohash,sxk9hm,40.987244,29.020386,KILISE_T3
713,geohash,sxk9hm,40.987244,29.020386,MODA ILKOKULU_T3
714,geohash,sxk9hm,40.987244,29.020386,MODA CADDESI_T3
716,geohash,sxk9hm,40.987244,29.020386,MUHURDAR_T3
718,geohash,sxk9hm,40.987244,29.020386,DAMGA SOKAK_T3
720,geohash,sxk9hm,40.987244,29.020386,KADIKOY IDO_T3


In [8]:
df.to_csv("datasets/13_gh_proximities/gh_nearby_metro_stns.csv", index=False)